In [ ]:
# Task 1: Load the libraries 

import os 
import io 
import warnings 
import pandas as pd 
import panel as pn 
import plotly.express as px 
from dotenv import load dot_env 
from langchain.llms import OpenAI
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent 

In [ ]:
# Run this cell after setting up the '.env' file
warnings.filterwarnings('ignore')
pn.extension('plotly', 'tabulator', comms="vscode")
load_dotenv()
print(os.environ['OPENAI_API_KEY'])

In [ ]:
# Task 2: The Initialization Method 

# Specify the path to the CSV file
file_name = 'data.csv'

In [ ]:
# Read the CSV file into a DataFrame and drop the 'id' column
data = pd.read_csv(file_name).drop(columns=['id'])

In [ ]:
# Create a Plotly pane for interactive plotting 
plot_pane = pn.pane.Plotly(sizing_mode="stretch_width")

In [ ]:
# Initialize a FileInput widget for uploading files
file_input = pn.widgets.FileInput()

In [ ]:
# Initialize a TextInput widget for user questions
text_input = pn.widgets.TextInput(
    name='Question',
    placeholder='Ask a question from the CSV',
    sizing_mode='scale_width'
)

In [ ]:
# Initialize a Button widget labeled 'Ask' for submitting questions
ask_button = pn.widgets.Button(
    name="Ask",
    button_type="primary",
    height=60
)

In [ ]:
# Initialize a Button widget labeled 'Load' for loading data
load_button = pn.widgets.Button(
    name="Load",
    button_type="primary"
)

In [ ]:
# Initialize a Button widget labeled 'Plot' for generating plots
plot_button = pn.widgets.Button(
    name="Plot",
    button_type="primary"
)

In [ ]:
# Initialize a ChatBox widget to display messages
chat_box = pn.chat.ChatInterface(
    callback=None, 
    user="User", 
    avatar="U", 
    callback_user="Assistant"
)

In [ ]:
# Task 3: Load the Page Components 

def load_page(data, file_name):
    # Identify the target variable (assumed to be the last column)
    target = data.columns[-1]

    # Dropdown for selecting the Y-axis variable, defaulting to the first numeric column
    yaxis = pn.widgets.Select(
        name='Y axis',
        options=list(data.columns),
        value=list(data._get_numeric_data().columns)[0],
        disabled_options=list(set(data.columns) - set(data._get_numeric_data().columns)),
    )

    # Dropdown for selecting the X-axis variable, defaulting to the second numeric column
    xaxis = pn.widgets.Select(
        name='X axis',
        options=list(data.columns),
        value=list(data._get_numeric_data().columns)[1],
        disabled_options=list(set(data.columns) - set(data._get_numeric_data().columns)),
    )

    # Create an initial scatter plot with the selected X and Y axes, colored by the target variable
    plot = px.scatter(
        data,
        x=list(data._get_numeric_data().columns)[0],
        y=list(data._get_numeric_data().columns)[1],
        color=target
    )

    # Create a table widget to display the DataFrame
    table = pn.widgets.Tabulator(data)

    # Initialize a CSV agent using LangChain's create_csv_agent function
    agent = create_csv_agent(
        OpenAI(model="gpt-3.5-turbo-instruct", temperature=0),
        file_name,
        verbose=True,
        return_intermediate_steps=True, 
        allow_dangerous_code=True
    )

    return target, yaxis, xaxis, plot, table, agent

In [ ]:
# Load the page components
target, yaxis, xaxis, plot_pane.object, table, agent = load_page(data, file_name)

In [ ]:
# Display the plot pane
plot_pane

In [ ]:
# Task 4: Construct the Web Application's Frontend 

# Create a FastListTemplate for the dashboard layout
template = pn.template.FastListTemplate(
    title='CSV-GPT',
    sidebar=[
        pn.pane.Markdown("# Ask your Data"),
        pn.pane.Markdown("## Upload your CSV File"),
        file_input,
        load_button,
        yaxis,
        xaxis,
        plot_button,
        plot_pane,
    ],
    main=[
        pn.Column(
            pn.pane.Markdown("## Data Preview"),
            table,
            height=300,
            sizing_mode="stretch_width",
            scroll=True
        ),
        pn.Column(
            pn.pane.Markdown("## Assistant Chat"),
            chat_box,
            height=500,
            scroll=True,
            sizing_mode="stretch_width"
        ),
        pn.Row(
            pn.Column(
                pn.pane.Markdown("## Ask a Question"),
                text_input,
                ask_button
            ),
            width=600
        )
        
    ],
    sidebar_width=420,
    accent_base_color="#88d8b0",
    header_background="#88d8b0"
)

In [ ]:
# Display the template in a web browser
# Note: Comment out this line after running the web app 
url = "localhost:5006"
template.show(open=False, address='0.0.0.0', port=5006, websocket_origin=url)

In [ ]:
def reloader(event):
    # Create a new scatter plot using Plotly Express with selected x and y axes
    plot_pane = px.scatter(data, x=xaxis.value, y=yaxis.value, color=target)
    
    # Update the plot in the sidebar with the new plot
    template.sidebar[6][0].object = plot_pane

# Attach the reloader function to the plot_button's on_click event
plot_button.on_click(reloader)

In [ ]:
def parse_file_input(event):
    global data, file_name

    # Retrieve the uploaded file’s content from the FileInput widget
    value = file_input.value
    # Convert the byte content into a BytesIO object
    bytes_io = io.BytesIO(value)
    # Read the CSV data into a pandas DataFrame
    data = pd.read_csv(bytes_io)
    # Get the filename of the uploaded file
    file_name = file_input.filename
    # Save the DataFrame to a CSV file with the original filename
    data.to_csv(file_name, index=False)

    # Declare global variables for UI components and agent
    global target, yaxis, xaxis, plot, table, agent

    # Initialize or update the UI components and agent based on the new data
    target, yaxis, xaxis, plot, table, agent = load_page(data, file_name)

    # Update the main template with the new table
    template.main[0][0] = table
    # Update the sidebar with the new y-axis selector
    template.sidebar[3][0] = yaxis
    # Update the sidebar with the new x-axis selector
    template.sidebar[4][0] = xaxis
    # Update the sidebar plot pane with the new plot
    template.sidebar[6][0].object = plot

In [ ]:
# Attach the parse_file_input function to the load_button‘s on_click event
load_button.on_click(parse_file_input)

In [ ]:
def ask(event):
    # Retrieve the user's query from the text input widget
    query = text_input.value
    # Append the user's query to the chat box
    chat_box.append({"User": query})
    # Send the query to the agent and get the response
    response = agent({"input": query})
    # Append the agent's thought process to the chat box
    chat_box.append({"Thought Process": [x[0].log for x in response["intermediate_steps"]]})
    # Append the agent's final response to the chat box
    chat_box.append({"Assistant": response['output']})

In [ ]:
# Bind the 'ask' function to the 'on_click' event of the ask button
ask_button.on_click(ask)